In [ ]:
# count number users
print(df['ID'].count())

# count defaults
print(df[df['default.payment.next.month']==1]['ID'].count())

# if the user has no bill payment at all
print(df[(df.BILL_AMT1==0) \
        &(df.BILL_AMT2==0) \
        &(df.BILL_AMT3==0) \
        &(df.BILL_AMT4==0) \
        &(df.BILL_AMT5==0) \
        &(df.BILL_AMT6==0) \
        & (df['default.payment.next.month']==1)
      ]['ID'].count())

# if the user pay more than their were ask to pay
print(df[(df.PAY_AMT1>=df.BILL_AMT1) \
        &(df.PAY_AMT2>=df.BILL_AMT2) \
        &(df.PAY_AMT3>=df.BILL_AMT3) \
        &(df.PAY_AMT4>=df.BILL_AMT4) \
        &(df.PAY_AMT5>=df.BILL_AMT5) \
        &(df.PAY_AMT6>=df.BILL_AMT6) \
        & (df['default.payment.next.month']==1)
      ]['ID'].count())

# if the user has bigger bill than their credit limit
print(df[((df.LIMIT_BAL<=df.BILL_AMT1) \
        |(df.LIMIT_BAL<=df.BILL_AMT2) \
        |(df.LIMIT_BAL<=df.BILL_AMT3) \
        |(df.LIMIT_BAL<=df.BILL_AMT4) \
        |(df.LIMIT_BAL<=df.BILL_AMT5) \
        |(df.LIMIT_BAL<=df.BILL_AMT6)) \
        & (df['default.payment.next.month']==1)
      ]['ID'].count())

Note that 
- the data dictionary the the README file is missing the item "PAY_1". I have, therefore, added this missing item into the data dictionary.
- "PAY_0" was defined as repayment status in September. I believe it was a mislabelled. "PAY_1" should be the status in September (consistent with "BILL_AMT#" and "PAY_AMT#). Then, "PAY_0" should be the status in October. 




In [ ]:

# To understand them, 
col_names = []
for i in range(1,6):
    col_names.append('BILL_AMT'+str(i))
    col_names.append('PAY_AMT'+str(i))
    col_names.append('PAY_'+str(i))

print(df_explore[np.array(col_names)].sample(10))




sns.distplot( df_explore.loc[df_explore.DEFAULT_PAY==0,'AGE'], color='skyblue', label='ok')
sns.distplot( df_explore.loc[df_explore.DEFAULT_PAY==1,'AGE'], color='red', label='default')










It is skewed though. I wonder what it would look like if I log transform it.

In [ ]:
select_cols = ['BILL_AMT'+str(i) for i in range(1,7)]
select_cols_log = ['BILL_AMT_LOG'+str(i) for i in range(1,7)]

# define function for the values below 0
def neg_to_zero(x):
    if x <= 0:
        return 1
    else:
        return x
    
# apply the function
for i in range(1,7):
    df_explore['BILL_AMT_LOG'+str(i)] = [neg_to_zero(x) for x in df_explore['BILL_AMT'+str(i)]]

#
df_explore[select_cols_log] = df_explore[select_cols].apply(np.log, axis = 1)

monthly_plot(df_explore, 'BILL_AMT_LOG')



continuous_names = \
    ['LIMIT_BAL', 'AGE', 'N_DELAYED', 'UTILIZE_PTG1', 'UTILIZE_PTG2', 'UTILIZE_PTG3',
       'UTILIZE_PTG4', 'UTILIZE_PTG5', 'UTILIZE_PTG6', 'PAY_PTG2',
       'PAY_PTG3', 'PAY_PTG4', 'PAY_PTG5', 'PAY_PTG6']


categorical_names = \
    ['SEX_male', 'EDUCATION_graduate_school', 'EDUCATION_high_school',
       'EDUCATION_university',
       'MARRIAGE_married', 'MARRIAGE_single', 'PAY_1_delayed']

In [ ]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=10,random_state=0,shuffle=True)

fold = []
scr = []

for i,(train_index, test_index) in enumerate(kf.split(df_onehot)):
    print(train_index)
    # split into K Fold sets
    train = df_onehot.iloc[train_index,:]
    test = df_onehot.iloc[test_index,:]
    # re-define features and labels data sets
    features_train = train[feature_names]
    labels_train = train['DEFAULT_PAY']
    features_test = test[feature_names]
    valid_label = valid['DEFAULT_PAY']
    # fit and predict
    clf.fit(features_train, labels_train)
    pred = clf.predict(features_test)
    # get f1 score
    score = f1_score(y_true = labels_test, y_pred = predictions)
    # store results
    fold.append(i+1)
    scr.append(score)
    
#create a small df with the scores
performance = pd.DataFrame({'Score':scr,'Fold':fold})

# show result of each fold
ax = sns.lineplot(x="Fold", y="Score",
                  marker="o",
                  markers=True, dashes=False, data=performance)

